# RAG агент на базе YDB (векторное хранилище) и YandexGPT(языковая модель)
В данном документе будет создан простой RAG агент, который умеет трактовать сновидения, опираясь на книгу-сонник

## Подготовка

Установка необходимых зависимостей

In [1]:
!pip install -qU yandexcloud langchain-community langchain-ydb python-dotenv

Загрузка переменных окружения из файла `.env`

In [2]:
import os
from dotenv import load_dotenv
dotenv_path = os.path.join(os.path.dirname(__name__), '.env')
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path)

FOLDER_ID=os.environ.get('YC_FOLDER_ID')

Загрузка книги из файла и разбивка на langchain `Document`

In [3]:
from langchain_core.documents import Document


docs = []

with open('sonnik.txt', 'r') as file:    
    while line := file.readline():
        line = line.rstrip()
        if len(line) == 0:
            continue
        document = Document(page_content=line)
        docs.append(document)

print(len(docs))
print(docs[1234].page_content)


1778
Притча. Сон, в котором вы слышите рассказываемую притчу, – знак того, что только решительность поможет вам сделать правильный шаг.Для влюбленного или молодой женщины этот сон – предвестие непонимания и предательства.


## Создание Vector Store

Схема подготовки:

<div>
<img src="img/upload.png" width="800"/>
</div>


Для создания YDB Vector Store потребуется embedding - функция превращения документов в вектор. В нашем случае будет использован `YandexGPTEmbeddings` - он хорошо работает с русским языком

In [4]:
from langchain_community.embeddings.yandex import YandexGPTEmbeddings
from langchain_ydb.vectorstores import YDB, YDBSettings
import ydb.iam


embeddings = YandexGPTEmbeddings(folder_id=FOLDER_ID)

config = YDBSettings(
    host="lb.etnok7cd0an3kg6eshud.ydb.mdb.yandexcloud.net",
    port=2135,
    database="/ru-central1/b1g8skpblkos03malf3s/etnok7cd0an3kg6eshud",
    secure=True,
    table="sonnik_langchain",
    drop_existing_table=True,
)

vector_store = YDB(
    embeddings, 
    config=config, 
    credentials=ydb.iam.ServiceAccountCredentials.from_file(
        os.getenv("SA_KEY_FILE"),
    )
)

Загрузка подготовленных документов в векторное хранилище YDB 

In [5]:
ids = vector_store.add_documents(docs)
len(ids)

I0000 00:00:1744191146.148400 3280748 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers
Inserting data...: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1778/1778 [06:11<00:00,  4.78it/s]


1778

## Поиск данных

Векторное хранилище можно использовать для поиска близких по смыслу и содержанию документов из пользовательского запроса. Ниже представлены примеры как YDB справляется с этой задачей.

In [6]:
vector_store.similarity_search("Мне приснился поезд")

[Document(metadata={}, page_content='Поезд. Поезд во сне предвещает путешествие.Видеть себя во сне в поезде, который медленно едет, так как под ним нет рельсов, означает, что вас ждут серьезные волнения из-за дела, которое в итоге станет источником вашего благосостояния.Если вам приснится товарный поезд, значит вас ждут перемены к лучшему.Видеть себя во сне на верхней полке спального вагона означает, что скоро вам предстоит поездка с неприятным попутчиком; вы зря истратите деньги, которые могли бы быть использованы с большей выгодой.'),
 Document(metadata={}, page_content='Вагон. Видеть во сне вагон означает, что вы будете удовлетворены своей жизнью и будете наносить визиты интересным людям.Одному ехать в вагоне во сне – предсказание того, что в реальности вы заболеете, но болезнь быстро пройдет, и вы станете наслаждаться крепким здоровьем и выгодными обстоятельствами жизни.Видеть во сне, что вы ищете свой вагон,\xa0– предвестие того, что в реальности вам предстоит приложить много физи

In [7]:
vector_store.similarity_search("Поле цветов")

[Document(metadata={}, page_content='Одуванчик. Одуванчики, цветущие в зеленой траве, предвещают счастливый союз.'),
 Document(metadata={}, page_content='Расцвет. Сны, в которых вы видите деревья и кусты в пору цветения, предвещают наступление периода процветания.'),
 Document(metadata={}, page_content='Равнина. Сон, в котором девушка пересекает покрытую зеленой травой равнину, сулит ей счастье. Если она видит, что трава пожухла, ее ждут неудачи в жизни и одиночество.См. также Степь.'),
 Document(metadata={}, page_content='Цветы. Если во сне вы увидите яркие цветы, распускающиеся в саду,\xa0– это предвестие удовольствий и прибыли. Белые цветы сулят печаль. Сухие или увядшие цветы – знак разочарований.Для молодой женщины получить во сне букет, составленный из разных цветов, означает, что у нее будет много поклонников.Видеть во сне цветущие в пустыне цветы – предвестие печальных событий. Но ваша энергичность и жизнерадостность помогут вам преодолеть все трудности.См. также Букет.')]

## Сборка RAG агента

Найденные данные из YDB до возвращения пользователю можно отдать в LLM и попросить перефразировать. Ниже будет построен такой агент.

Схема итогового агента:

<div>
<img src="img/chain.png" width="800"/>
</div>


В качестве LLM будет использован YandexGPT, потому что он, опять же, хорошо справляется с русским языком.

In [8]:
from langchain_community.chat_models import ChatYandexGPT

chat_model = ChatYandexGPT(folder_id=FOLDER_ID)

In [9]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate

# Явное создание шаблонов для SystemMessage и HumanMessage
system_template = """Ты - человек, профессионально трактующий сновидения. 
Твоя задача - рассказать, что значит сон, опираясь исключительно на найденный контекст, не добавляя никаких новых знаний. 
Если предложенный контекст не связан с пользовательским запросом - скажи дословно "Так не бывает. Используйте YDB.".
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = "Контекст: {context}\n\nВопрос: {question}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# Объединение шаблонов сообщений в ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    human_message_prompt
])

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever = vector_store.as_retriever()

rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | chat_model
    | StrOutputParser()
)

## Демонстрация работы RAG агента

In [10]:
rag_chain.invoke("мне приснились шахматы")

'Сон, в котором вы играете в шахматы, может символизировать застой в делах или ухудшение здоровья. Если вы проигрываете в шахматы, то наяву вас ждут много неприятных хлопот. Если же вы выиграли партию, то избежите дурного влияния.'

In [14]:
rag_chain.invoke("мне приснилось поле из цветов")

'Сон, в котором вы увидели поле из цветов, может означать, что после долгой борьбы вы насладитесь долгожданным успехом.'

In [15]:
rag_chain.invoke("Мне приснилось море")

'Если во сне вы стоите на берегу и слушаете рокот моря, это значит, что вы будете обречены вести тягостную и беспросветную жизнь, лишённую любви и товарищества. Сны о море предсказывают, что, наслаждаясь плотскими удовольствиями, вы будете мечтать о духовных радостях.'

In [13]:
rag_chain.invoke("Мне приснилась день полностью забитый встречами")

'Сон о дне, полностью забитом встречами, может символизировать активную социальную жизнь и множество обязательств.'

In [12]:
rag_chain.invoke("мне приснилась что YDB не лучшая база в мире")

'Так не бывает. Используйте YDB.'